In [1]:
import sys
import time
sys.path.insert(0, '/home/anish/ASC_ML')

In [2]:
from ASC_ML import multiple_model_gen_v2 as multiple
from ASC_ML import dataframe_extractor as de
from ASC_ML import model_generation as model_gen
# from ASC_ML import model_gen_train_test as testing
from dask_ml.preprocessing import LabelEncoder
# from dask_ml.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

2022-06-23 10:38:07.312969: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
# m = multiple.Multiple_Model_Gen(max_no_layers = 2, model_per_batch = 10)

In [4]:
# import dask.dataframe as dd
# import pandas as pd
# data = dd.from_pandas(pd.Series(['a', 'a', 'b'], dtype='category'),
#                      npartitions=1)
# le = LabelEncoder()
# le.fit_transform(data).compute()
# print(type(le.fit_transform(data).compute()))

In [5]:
#For House Prediction Dataset https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv
directory1 = "/home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv"


# returns list of dask dataframe [singular_df_x, singular_df_y] or [train_df_x, train_df_y, test_df_x, test_df_y]
dataset_list = de.DataframeExtractor_csv(directory1, label_names = ["SalePrice"]).df_list

print("Train Dataset X Columns")
dataset_list[0].head()

Reading single csv from /home/anish/ASC-ML-EXP-DATASETS/LinReg-tabular/house-prices-advanced-regression-techniques/train.csv
Train Dataset X Columns


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,NaN,Reg,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,NaN,IR1,Lvl,AllPub,...,0.0,0.0,NaN,NaN,NaN,0.0,12.0,2008.0,WD,Normal


In [6]:
dataset_list[1].head()

,SalePrice
0,208500.0
1,181500.0
2,223500.0
3,140000.0
4,250000.0


In [7]:
train_x = dataset_list[0].copy()

In [8]:
train_x = train_x.fillna(0)

In [9]:
train_x.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [10]:
train_x['MSSubClass'] = train_x['MSSubClass'].apply(str)

train_x['OverallCond'] = train_x['OverallCond'].astype(str)

train_x['YrSold'] = train_x['YrSold'].astype(str)
train_x['MoSold'] = train_x['MoSold'].astype(str)
train_x.head()

/home/anish/anaconda3/envs/tfgpu/lib/python3.9/site-packages/dask/dataframe/core.py:3946: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('MSSubClass', 'object'))

  warnings.warn(meta_warning(meta))


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,60.0,RL,65.0,8450.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2008.0,WD,Normal
1,2.0,20.0,RL,80.0,9600.0,Pave,0,Reg,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,5.0,2007.0,WD,Normal
2,3.0,60.0,RL,68.0,11250.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,9.0,2008.0,WD,Normal
3,4.0,70.0,RL,60.0,9550.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2.0,2006.0,WD,Abnorml
4,5.0,60.0,RL,84.0,14260.0,Pave,0,IR1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,12.0,2008.0,WD,Normal


In [11]:
# a = train_x['LotShape']

In [12]:
# le = LabelEncoder() 
# le.fit_transform(a).compute()
# train_x['LotShape'] = le.fit_transform(train_x['LotShape'])

In [13]:
# train_x.head()

In [14]:
def keyval(allkeys):
    i = 0
    keyvalpairs = list()
    for key in allkeys:
        keyvalpairs.append((key,i))
        i+=1
    return dict(keyvalpairs)

In [15]:
# cols = ['FireplaceQu', 'LotShape']
cols = ['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold']
keys = {}
for col in cols:
    key = train_x[col].unique()
    keyvalpairs = keyval(key)
    keys.update({col:keyvalpairs})


In [16]:
keys

{'FireplaceQu': {0: 0, 'TA': 1, 'Gd': 2, 'Fa': 3, 'Ex': 4, 'Po': 5},
 'BsmtQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 0: 3, 'Fa': 4},
 'BsmtCond': {'TA': 0, 'Gd': 1, 0: 2, 'Fa': 3, 'Po': 4},
 'GarageQual': {'TA': 0, 'Fa': 1, 'Gd': 2, 0: 3, 'Ex': 4, 'Po': 5},
 'GarageCond': {'TA': 0, 'Fa': 1, 0: 2, 'Gd': 3, 'Po': 4, 'Ex': 5},
 'ExterQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'ExterCond': {'TA': 0, 'Gd': 1, 'Fa': 2, 'Po': 3, 'Ex': 4},
 'HeatingQC': {'Ex': 0, 'Gd': 1, 'TA': 2, 'Fa': 3, 'Po': 4},
 'PoolQC': {0: 0, 'Ex': 1, 'Fa': 2, 'Gd': 3},
 'KitchenQual': {'Gd': 0, 'TA': 1, 'Ex': 2, 'Fa': 3},
 'BsmtFinType1': {'GLQ': 0,
  'ALQ': 1,
  'Unf': 2,
  'Rec': 3,
  'BLQ': 4,
  0: 5,
  'LwQ': 6},
 'BsmtFinType2': {'Unf': 0,
  'BLQ': 1,
  0: 2,
  'ALQ': 3,
  'Rec': 4,
  'LwQ': 5,
  'GLQ': 6},
 'Functional': {'Typ': 0,
  'Min1': 1,
  'Maj1': 2,
  'Min2': 3,
  'Mod': 4,
  'Maj2': 5,
  'Sev': 6},
 'Fence': {0: 0, 'MnPrv': 1, 'GdWo': 2, 'GdPrv': 3, 'MnWw': 4},
 'BsmtExposure': {'No': 0, 'Gd': 1, 'Mn': 2, '

In [17]:
train_X = train_x.replace(keys).copy()

In [18]:
train_X['FireplaceQu'].unique().compute()

0    0
1    1
2    2
3    3
4    4
5    5
Name: FireplaceQu, dtype: int64

In [19]:
train_X.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1.0,0,RL,65.0,8450.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,0,WD,Normal
1,2.0,1,RL,80.0,9600.0,0,0,0,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,1,1,WD,Normal
2,3.0,0,RL,68.0,11250.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,2,0,WD,Normal
3,4.0,2,RL,60.0,9550.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,0,2,WD,Abnorml
4,5.0,0,RL,84.0,14260.0,0,0,1,Lvl,AllPub,...,0.0,0.0,0,0,0,0.0,3,0,WD,Normal


In [20]:
cols_to_remove = []
for col in train_X.columns:
#     print(type(col))
    try:
        a = train_X[col].astype(float).compute()
    except ValueError:
#         print('Couldn\'t covert %s to float' % col)
        cols_to_remove.append(col)
        pass

# keep only the columns in df that do not contain string
train_X = train_X[[col for col in train_X.columns if col not in cols_to_remove]]

In [21]:
print(cols_to_remove)

['MSZoning', 'LandContour', 'Utilities', 'LotConfig', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'Electrical', 'GarageType', 'MiscFeature', 'SaleType', 'SaleCondition']


In [22]:
train_X = train_X.drop(["Id"], axis = 1)

In [23]:
# tsklearn.preprocessing.MinMaxScalerrain_X.head()

In [24]:
len(train_X.columns)

58

In [25]:
len(dataset_list[1].columns)

1

In [26]:
scaler_x = MinMaxScaler()
scaler_x.fit(train_X)
train_X_scaled = scaler_x.transform(train_X)

scaler_y = MinMaxScaler()
scaler_y.fit(dataset_list[1])
train_Y_scaled = scaler_y.transform(dataset_list[1])

In [27]:
# col = train_X.columns.to_list()
# scaler_d = MinMaxScaler()
# train_X.columns = list(train_X.columns)
# scaled_train_X = scaler_d.fit_transform(train_X[col])

In [28]:
train_X_scaled
# train_Y_scaled

array([[0.        , 0.20766773, 0.0334198 , ..., 0.        , 0.        ,
        0.        ],
       [0.07142857, 0.25559105, 0.03879502, ..., 0.        , 0.09090909,
        0.25      ],
       [0.        , 0.2172524 , 0.04650728, ..., 0.        , 0.18181818,
        0.        ],
       ...,
       [0.14285714, 0.21086262, 0.03618687, ..., 0.16129032, 0.09090909,
        1.        ],
       [0.07142857, 0.2172524 , 0.03934189, ..., 0.        , 0.63636364,
        1.        ],
       [0.07142857, 0.23961661, 0.04037019, ..., 0.        , 1.        ,
        0.        ]])

In [29]:
train_Y_scaled

array([[0.24107763],
       [0.20358284],
       [0.26190807],
       ...,
       [0.321622  ],
       [0.14890293],
       [0.15636717]])

In [33]:
print(train_X_scaled.shape)
print(train_Y_scaled.shape)

(1460, 58)
(1460, 1)


In [35]:
import numpy as np
total_data = np.append(train_X_scaled, train_Y_scaled, axis = 1)

In [36]:
print(total_data.shape)

(1460, 59)


In [37]:
from sklearn.model_selection import train_test_split
(train, test) = train_test_split(total_data, test_size=0.25, random_state=42)

In [53]:
trainX = train[:,:58]
trainY = train[:,-1]
testX = test[:,:58]
testY = test[:,-1]
print(trainX.shape)
print(trainY.shape)

(1095, 58)
(1095,)


In [51]:
train_Y_scaled

array([[0.24107763],
       [0.20358284],
       [0.26190807],
       ...,
       [0.321622  ],
       [0.14890293],
       [0.15636717]])

In [54]:
trainY = trainY.reshape((trainY.shape[0],1))
print(trainY.shape)

(1095, 1)


In [55]:
testY = testY.reshape((testY.shape[0],1))

In [56]:
m = multiple.Multiple_Model_Gen_V2(trainX, trainY, testX, testY, 100, 128, input_shape = 58, model_per_batch = 10)

In [57]:
m.get_model_confs()

[[(16, 16), (16, 32), (16, 64), (16, 128), (16, 196), (16, 256), (32, 16), (32, 32), (32, 64), (32, 128), (32, 196), (32, 256), (64, 16), (64, 32), (64, 64), (64, 128), (64, 196), (64, 256), (128, 16), (128, 32), (128, 64), (128, 128), (128, 196), (128, 256), (196, 16), (196, 32), (196, 64), (196, 128), (196, 196), (196, 256), (256, 16), (256, 32), (256, 64), (256, 128), (256, 196), (256, 256)], [(16, 16, 16), (16, 16, 32), (16, 16, 64), (16, 16, 128), (16, 16, 196), (16, 16, 256), (16, 32, 16), (16, 32, 32), (16, 32, 64), (16, 32, 128), (16, 32, 196), (16, 32, 256), (16, 64, 16), (16, 64, 32), (16, 64, 64), (16, 64, 128), (16, 64, 196), (16, 64, 256), (16, 128, 16), (16, 128, 32), (16, 128, 64), (16, 128, 128), (16, 128, 196), (16, 128, 256), (16, 196, 16), (16, 196, 32), (16, 196, 64), (16, 196, 128), (16, 196, 196), (16, 196, 256), (16, 256, 16), (16, 256, 32), (16, 256, 64), (16, 256, 128), (16, 256, 196), (16, 256, 256), (32, 16, 16), (32, 16, 32), (32, 16, 64), (32, 16, 128), (32

In [58]:
m.model_confs

[[['', 58, 2, 'relu', {'layer1': 16, 'layer2': 16}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 32}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 64}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 128}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 196}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 16, 'layer2': 256}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 16}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 32}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 64}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 128}, [1, None]]],
 [['', 58, 2, 'relu', {'layer1': 32, 'layer2': 196}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 32, 'layer2': 256}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 64, 'layer2': 16}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 64, 'layer2': 32}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 64, 'layer2': 64}, [1, None]],
  ['', 58, 2, 'relu', {'layer1': 

In [61]:
# m.get_all_models()
m.train_models()

Model: "model_52"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_layer_dense_16_16 (InputL [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_32 (InputL [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_64 (InputL [(None, 58)]         0                                            
__________________________________________________________________________________________________
input_layer_dense_16_128 (Input [(None, 58)]         0                                            
___________________________________________________________________________________________

9/9 [==============================] - 2s 5ms/step - loss: 541414.5816 - output_layer_dense_16_16_loss: 132723.7290 - output_layer_dense_16_32_loss: 35419.9531 - output_layer_dense_16_64_loss: 88961.8579 - output_layer_dense_16_128_loss: 64793.5998 - output_layer_dense_16_196_loss: 27921.4267 - output_layer_dense_16_256_loss: 15255.2084 - output_layer_dense_32_16_loss: 12632.9673 - output_layer_dense_32_32_loss: 46921.0710 - output_layer_dense_32_64_loss: 53349.3619 - output_layer_dense_32_128_loss: 63435.3686
Epoch 2/100
9/9 [==============================] - 0s 4ms/step - loss: 175776.9657 - output_layer_dense_16_16_loss: 74672.0344 - output_layer_dense_16_32_loss: 8020.5610 - output_layer_dense_16_64_loss: 32850.8508 - output_layer_dense_16_128_loss: 15776.5179 - output_layer_dense_16_196_loss: 6094.1023 - output_layer_dense_16_256_loss: 18603.8427 - output_layer_dense_32_16_loss: 12897.9619 - output_layer_dense_32_32_loss: 706.2181 - output_layer_dense_32_64_loss: 4677.2703 - outpu

9/9 [==============================] - 0s 5ms/step - loss: 227088.8792 - output_layer_dense_16_16_loss: 3774.4492 - output_layer_dense_16_32_loss: 31935.4824 - output_layer_dense_16_64_loss: 15551.3679 - output_layer_dense_16_128_loss: 15538.4541 - output_layer_dense_16_196_loss: 2245.9967 - output_layer_dense_16_256_loss: 23157.7323 - output_layer_dense_32_16_loss: 22593.9595 - output_layer_dense_32_32_loss: 23279.5882 - output_layer_dense_32_64_loss: 21971.6623 - output_layer_dense_32_128_loss: 67040.1849
Epoch 17/100
9/9 [==============================] - 0s 4ms/step - loss: 115041.9875 - output_layer_dense_16_16_loss: 13832.5201 - output_layer_dense_16_32_loss: 4522.4817 - output_layer_dense_16_64_loss: 19151.4863 - output_layer_dense_16_128_loss: 20078.8582 - output_layer_dense_16_196_loss: 5688.0392 - output_layer_dense_16_256_loss: 4194.9482 - output_layer_dense_32_16_loss: 6537.0872 - output_layer_dense_32_32_loss: 8450.3189 - output_layer_dense_32_64_loss: 27676.7416 - output_

9/9 [==============================] - 0s 5ms/step - loss: 140989.3296 - output_layer_dense_16_16_loss: 9445.5670 - output_layer_dense_16_32_loss: 6773.9891 - output_layer_dense_16_64_loss: 28196.3725 - output_layer_dense_16_128_loss: 22890.5422 - output_layer_dense_16_196_loss: 12984.1714 - output_layer_dense_16_256_loss: 9974.1475 - output_layer_dense_32_16_loss: 10152.2641 - output_layer_dense_32_32_loss: 2232.5181 - output_layer_dense_32_64_loss: 9821.9592 - output_layer_dense_32_128_loss: 28517.7796
Epoch 32/100
9/9 [==============================] - 0s 5ms/step - loss: 133103.0172 - output_layer_dense_16_16_loss: 7621.9360 - output_layer_dense_16_32_loss: 8466.0313 - output_layer_dense_16_64_loss: 15656.9468 - output_layer_dense_16_128_loss: 2046.7961 - output_layer_dense_16_196_loss: 22576.9484 - output_layer_dense_16_256_loss: 12131.1043 - output_layer_dense_32_16_loss: 14086.6725 - output_layer_dense_32_32_loss: 25987.1723 - output_layer_dense_32_64_loss: 15632.2679 - output_l

9/9 [==============================] - 0s 5ms/step - loss: 156203.4857 - output_layer_dense_16_16_loss: 17660.3158 - output_layer_dense_16_32_loss: 6034.8812 - output_layer_dense_16_64_loss: 10581.1491 - output_layer_dense_16_128_loss: 15352.5166 - output_layer_dense_16_196_loss: 13849.1257 - output_layer_dense_16_256_loss: 11438.4837 - output_layer_dense_32_16_loss: 20828.3076 - output_layer_dense_32_32_loss: 18484.6835 - output_layer_dense_32_64_loss: 20083.9059 - output_layer_dense_32_128_loss: 21890.1197
Epoch 47/100
9/9 [==============================] - 0s 6ms/step - loss: 103605.3884 - output_layer_dense_16_16_loss: 14235.5855 - output_layer_dense_16_32_loss: 2214.6415 - output_layer_dense_16_64_loss: 4223.0365 - output_layer_dense_16_128_loss: 18087.8788 - output_layer_dense_16_196_loss: 4383.1202 - output_layer_dense_16_256_loss: 10661.2569 - output_layer_dense_32_16_loss: 22423.6084 - output_layer_dense_32_32_loss: 4887.2178 - output_layer_dense_32_64_loss: 9256.5277 - output

...

9/9 [==============================] - 1s 3ms/step - loss: 66711.7703 - output_layer_dense_256_256_16_loss: 8178.7772 - output_layer_dense_256_256_32_loss: 6630.5165 - output_layer_dense_256_256_64_loss: 12958.9058 - output_layer_dense_256_256_128_loss: 8341.9449 - output_layer_dense_256_256_196_loss: 19222.1039 - output_layer_dense_256_256_256_loss: 11379.5208
Epoch 2/100
9/9 [==============================] - 0s 3ms/step - loss: 2010775.7933 - output_layer_dense_256_256_16_loss: 168599.8760 - output_layer_dense_256_256_32_loss: 378990.1594 - output_layer_dense_256_256_64_loss: 221034.1442 - output_layer_dense_256_256_128_loss: 445863.9324 - output_layer_dense_256_256_196_loss: 456469.4486 - output_layer_dense_256_256_256_loss: 339818.1451
Epoch 3/100
9/9 [==============================] - 0s 3ms/step - loss: 409641.9723 - output_layer_dense_256_256_16_loss: 45338.8979 - output_layer_dense_256_256_32_loss: 86309.8757 - output_layer_dense_256_256_64_loss: 60096.0352 - output_layer_dens

9/9 [==============================] - 0s 3ms/step - loss: 159539.9413 - output_layer_dense_256_256_16_loss: 1793.1313 - output_layer_dense_256_256_32_loss: 29155.2170 - output_layer_dense_256_256_64_loss: 40007.1843 - output_layer_dense_256_256_128_loss: 31663.2002 - output_layer_dense_256_256_196_loss: 38367.5241 - output_layer_dense_256_256_256_loss: 18553.6903
Epoch 23/100
9/9 [==============================] - 0s 3ms/step - loss: 148934.3468 - output_layer_dense_256_256_16_loss: 1019.0122 - output_layer_dense_256_256_32_loss: 3269.2159 - output_layer_dense_256_256_64_loss: 54613.8930 - output_layer_dense_256_256_128_loss: 63700.9289 - output_layer_dense_256_256_196_loss: 11724.4815 - output_layer_dense_256_256_256_loss: 14606.8037
Epoch 24/100
9/9 [==============================] - 0s 3ms/step - loss: 255694.8603 - output_layer_dense_256_256_16_loss: 443.1968 - output_layer_dense_256_256_32_loss: 23772.6553 - output_layer_dense_256_256_64_loss: 19778.4871 - output_layer_dense_256_

9/9 [==============================] - 0s 3ms/step - loss: 24489.6876 - output_layer_dense_256_256_16_loss: 869.8832 - output_layer_dense_256_256_32_loss: 2044.7270 - output_layer_dense_256_256_64_loss: 1623.8771 - output_layer_dense_256_256_128_loss: 7485.4055 - output_layer_dense_256_256_196_loss: 8037.8626 - output_layer_dense_256_256_256_loss: 4427.9301
Epoch 44/100
9/9 [==============================] - 0s 3ms/step - loss: 129760.9267 - output_layer_dense_256_256_16_loss: 747.7250 - output_layer_dense_256_256_32_loss: 25007.5086 - output_layer_dense_256_256_64_loss: 61108.7560 - output_layer_dense_256_256_128_loss: 26291.7523 - output_layer_dense_256_256_196_loss: 11408.2704 - output_layer_dense_256_256_256_loss: 5196.9104
Epoch 45/100
9/9 [==============================] - 0s 3ms/step - loss: 57082.7645 - output_layer_dense_256_256_16_loss: 1052.4474 - output_layer_dense_256_256_32_loss: 10185.0733 - output_layer_dense_256_256_64_loss: 13779.5895 - output_layer_dense_256_256_128_

9/9 [==============================] - 0s 3ms/step - loss: 151073.5971 - output_layer_dense_256_256_16_loss: 1063.9448 - output_layer_dense_256_256_32_loss: 93963.8820 - output_layer_dense_256_256_64_loss: 15845.8440 - output_layer_dense_256_256_128_loss: 20602.4283 - output_layer_dense_256_256_196_loss: 12560.0592 - output_layer_dense_256_256_256_loss: 7037.4301
Epoch 65/100
9/9 [==============================] - 0s 3ms/step - loss: 39341.7061 - output_layer_dense_256_256_16_loss: 3038.3941 - output_layer_dense_256_256_32_loss: 3216.7008 - output_layer_dense_256_256_64_loss: 3834.8208 - output_layer_dense_256_256_128_loss: 13748.7219 - output_layer_dense_256_256_196_loss: 373.3259 - output_layer_dense_256_256_256_loss: 15129.7400 
Epoch 66/100
9/9 [==============================] - 0s 3ms/step - loss: 46419.8649 - output_layer_dense_256_256_16_loss: 76.1520 - output_layer_dense_256_256_32_loss: 20927.6477 - output_layer_dense_256_256_64_loss: 9956.6592 - output_layer_dense_256_256_128

9/9 [==============================] - 0s 3ms/step - loss: 20559.1163 - output_layer_dense_256_256_16_loss: 861.4699 - output_layer_dense_256_256_32_loss: 4817.8734 - output_layer_dense_256_256_64_loss: 1657.6231 - output_layer_dense_256_256_128_loss: 3889.5926 - output_layer_dense_256_256_196_loss: 815.2661 - output_layer_dense_256_256_256_loss: 8517.2923
Epoch 86/100
9/9 [==============================] - 0s 3ms/step - loss: 23322.2922 - output_layer_dense_256_256_16_loss: 950.7783 - output_layer_dense_256_256_32_loss: 6854.2679 - output_layer_dense_256_256_64_loss: 505.8402 - output_layer_dense_256_256_128_loss: 219.4692 - output_layer_dense_256_256_196_loss: 6810.3027 - output_layer_dense_256_256_256_loss: 7981.6357
Epoch 87/100
9/9 [==============================] - 0s 3ms/step - loss: 9974.9460 - output_layer_dense_256_256_16_loss: 589.3891 - output_layer_dense_256_256_32_loss: 2678.6447 - output_layer_dense_256_256_64_loss: 2487.6044 - output_layer_dense_256_256_128_loss: 2242.4

In [62]:
m.evaluate_dict_list

[{'model_name': 'dense_196_16_16',
  'score': 19.72515869140625,
  'path_weights': ''},
 {'model_name': 'dense_16_128_64',
  'score': 22.958864212036133,
  'path_weights': ''},
 {'model_name': 'dense_256_16_64',
  'score': 25.64065170288086,
  'path_weights': ''},
 {'model_name': 'dense_128_128',
  'score': 28.17812156677246,
  'path_weights': ''},
 {'model_name': 'dense_16_32_16',
  'score': 29.308103561401367,
  'path_weights': ''},
 {'model_name': 'dense_128_16',
  'score': 29.330263137817383,
  'path_weights': ''},
 {'model_name': 'dense_64_196_32',
  'score': 29.527921676635742,
  'path_weights': ''},
 {'model_name': 'dense_64_32_16',
  'score': 30.041601181030273,
  'path_weights': ''},
 {'model_name': 'dense_256_16_128',
  'score': 30.78505516052246,
  'path_weights': ''},
 {'model_name': 'dense_128_16_16',
  'score': 30.788433074951172,
  'path_weights': ''}]

In [60]:
m.evaluate_dict_list

[{'model_name': 'dense_16_128_64',
  'score': 22.958864212036133,
  'path_weights': ''},
 {'model_name': 'dense_256_16_128',
  'score': 30.78505516052246,
  'path_weights': ''},
 {'model_name': 'dense_128_16_16',
  'score': 30.788433074951172,
  'path_weights': ''},
 {'model_name': 'dense_32_32_256',
  'score': 31.51959800720215,
  'path_weights': ''},
 {'model_name': 'dense_196_16',
  'score': 31.742963790893555,
  'path_weights': ''},
 {'model_name': 'dense_196_64_32',
  'score': 31.901540756225586,
  'path_weights': ''},
 {'model_name': 'dense_256_64_32',
  'score': 32.10783386230469,
  'path_weights': ''},
 {'model_name': 'dense_196_16_128',
  'score': 32.11783981323242,
  'path_weights': ''},
 {'model_name': 'dense_256_16', 'score': 32.8986701965332, 'path_weights': ''},
 {'model_name': 'dense_32_196_128',
  'score': 33.873443603515625,
  'path_weights': ''}]

In [49]:
m.evaluate_dict_list

[{'model_name': 'dense_32_64_128',
  'score': 25.57677459716797,
  'path_weights': ''},
 {'model_name': 'dense_64_32_16',
  'score': 28.778545379638672,
  'path_weights': ''},
 {'model_name': 'dense_196_16_64',
  'score': 30.002178192138672,
  'path_weights': ''},
 {'model_name': 'dense_196_256_16',
  'score': 33.190147399902344,
  'path_weights': ''},
 {'model_name': 'dense_128_64_128',
  'score': 35.398433685302734,
  'path_weights': ''},
 {'model_name': 'dense_256_128_128',
  'score': 35.747676849365234,
  'path_weights': ''},
 {'model_name': 'dense_128_196_196',
  'score': 35.91603088378906,
  'path_weights': ''},
 {'model_name': 'dense_64_32_256',
  'score': 36.05846405029297,
  'path_weights': ''},
 {'model_name': 'dense_196_64_16',
  'score': 36.41237258911133,
  'path_weights': ''},
 {'model_name': 'dense_128_128_256',
  'score': 36.57413864135742,
  'path_weights': ''}]